In [1]:
using Knet

STANFORD GLOVE EMBEDDINGS 

In [2]:
f = open("glove.42B.300d.txt")

IOStream(<file glove.42B.300d.txt>)

In [3]:
lines = readlines(f);

In [5]:
embeddingdict = Dict()
embeddingindex = 1
for line in lines
    strword = split(line)
    strname = strword[1]
    embeddingdict[strname] = embeddingindex
    embeddingindex+=1
end

SENTIMENT TREEBANK DATA

In [161]:
lowercase("The")

"the"

In [227]:
txt1 = open("sentences.txt")
lines1 = readlines(txt1)
txt2 = open("labels.txt")
lines2 = readlines(txt2);

In [228]:
all_sentences = (x -> split(x)).(lines1);
all_sentences = [(x -> lowercase(x)).(s) for s in all_sentences]
        
train_sentences2 = all_sentences[1:8534]
test_sentences2 = all_sentences[8535:10744]
val_sentences2 = all_sentences[10745:11844]

train_labels = lines2[1:8534]
test_labels = lines2[8535:10744]
val_labels = lines2[10745:11844];

In [183]:
vocab = []
for sentence in all_sentences
    for word in sentence
        if !(word in vocab); push!(vocab, word);end
    end
end

In [229]:
w2i = Dict()
i2w = Dict()
dictindex = 1
for word in vocab
    w2i[word] = dictindex
    i2w[dictindex] = word
    dictindex+=1
end

In [230]:
all_sentences = [(x -> w2i[x]).(s) for s in all_sentences]
train_sentences = all_sentences[1:8534]
test_sentences = all_sentences[8535:10744]
val_sentences = all_sentences[10745:11844];

CONSTRUCTING THE EMBEDDING MATRIX

In [244]:
embed = []
count = 0
for word in vocab
    if word in keys(embeddingdict)
        wordvector = (x-> parse(Float32, x)).(split(lines[embeddingdict[word]])[2:301])
        count += 1
    else
        wordvector = zeros(Float32, 300)
    end
    push!(embed, hcat(wordvector))
end

In [245]:
embed = KnetArray(permutedims(hcat(embed...)));

In [246]:
size(embed)

(19506, 300)

In [247]:
println(count, " out of ", length(vocab), " words are in Stanford Glove Embeddings. The rest is initialized randomly.")

18593 out of 19506 words are in Stanford Glove Embeddings. The rest is initialized randomly.


In [248]:
#CHECK that it works
checker = []
for line in lines
    sentence = split(line)
    if sentence[1] == "terrible"
        checker = (sentence[2:301])
        break;
    end
end
parse(Float32, checker[1]) == embed[w2i["terrible"]]

true

SET UP MODEL

In [253]:
# Set constants for the model and training
EPOCHS=3          # Number of training epochs
BATCHSIZE=34      # Number of instances in a minibatch
EMBEDSIZE=125     # Word embedding size
NUMHIDDEN=100     # Hidden layer size
MAXLEN=150        # maximum size of the word sequence, pad shorter sequences, truncate longer ones
VOCABSIZE=30000   # maximum vocabulary size, keep the most frequent 30K, map the rest to UNK token
NUMCLASS=2        # number of output classes
DROPOUT=0.5       # Dropout rate
LR=0.001          # Learning rate
BETA_1=0.9        # Adam optimization parameter
BETA_2=0.999      # Adam optimization parameter
EPS=1e-08         # Adam optimization parameter

1.0e-8

In [259]:
struct SequenceClassifier; input; rnn; output; pdrop; end

In [260]:
SequenceClassifier(input::Int, embed::Int, hidden::Int, output::Int; pdrop=0) =
    SequenceClassifier(param(embed,input), RNN(embed,hidden,rnnType=:gru), param(output,hidden), pdrop)

SequenceClassifier

In [261]:
SequenceClassifier(input,rnn,output;pdrop=0) = SequenceClassifier(input,rnn,output,pdrop)

SequenceClassifier

In [262]:
function (sc::SequenceClassifier)(input)
    embed = sc.input[:, permutedims(hcat(input...))]
    embed = dropout(embed,sc.pdrop)
    hidden = sc.rnn(embed)
    hidden = dropout(hidden,sc.pdrop)
    return sc.output * hidden[:,:,end]
end

In [267]:
(sc::SequenceClassifier)(input,output) = nll(sc(input),output)
(sc::SequenceClassifier)(d::Knet.Data) = Knet.mean(sc(x,y) for (x,y) in d)

In [264]:
dtrn = minibatch(train_sentences,train_labels,BATCHSIZE;shuffle=true)
dtst = minibatch(test_sentences,test_labels ,BATCHSIZE)
length(dtrn), length(dtst)

(251, 65)

In [265]:
model = SequenceClassifier(VOCABSIZE,EMBEDSIZE,NUMHIDDEN,NUMCLASS,pdrop=DROPOUT)

SequenceClassifier(P(KnetArray{Float32,2}(125,30000)), GRU(input=125,hidden=100), P(KnetArray{Float32,2}(2,100)), 0.5)

In [268]:
model(dtrn)

DimensionMismatch: DimensionMismatch("vectors must have same lengths")